# Model Performance Metrics

This notebook analyzes model performance with focus on efficiency metrics:
1. Memory usage
2. Inference time
3. Model size
4. Efficiency scores

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from utils import (
    measure_model_metrics,
    calculate_efficiency_score,
    get_model_summary,
    optimize_batch_size,
)

# Set style for better visualizations
plt.style.use("seaborn")
sns.set_palette("husl")

OSError: 'seaborn' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

## 1. Load Models and Data

In [ ]:
# Load test data
X_test = np.load("preprocessed/X_test.npy")
y_test = np.load("preprocessed/y_test.npy")
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Load all models
models = {}
for model_file in os.listdir("models"):
    if model_file.endswith(".h5"):
        name = model_file[:-3]
        models[name] = tf.keras.models.load_model(f"models/{model_file}")

print(f"Loaded {len(models)} models:")
for name in models.keys():
    print(f"- {name}")

## 2. Measure Performance Metrics

In [ ]:
# Measure metrics for each model
metrics = {}

for name, model in models.items():
    print(f"\nAnalyzing {name}...")
    
    # Measure performance metrics
    metrics[name] = measure_model_metrics(model, X_test_reshaped)
    
    # Get model summary
    metrics[name]['summary'] = get_model_summary(model, metrics[name])
    
    # Find optimal batch size
    metrics[name]['optimal_batch'] = optimize_batch_size(model, X_test_reshaped)
    
    # Calculate predictions and accuracy
    y_pred = np.argmax(model.predict(X_test_reshaped), axis=1)
    accuracy = np.mean(y_pred == y_test)
    metrics[name]['accuracy'] = accuracy
    
    # Calculate efficiency score
    metrics[name]['efficiency_score'] = calculate_efficiency_score(
        accuracy,
        metrics[name]['model_size'],
        metrics[name]['inference_time']['mean']
    )
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Efficiency Score: {metrics[name]['efficiency_score']:.4f}")

## 3. Compare Model Sizes

In [ ]:
# Plot model sizes
sizes = {name: m['model_size'] for name, m in metrics.items()}
plt.figure(figsize=(10, 6))
bars = plt.bar(sizes.keys(), sizes.values())

# Color hybrid models differently
for i, bar in enumerate(bars):
    if any(x in list(sizes.keys())[i] for x in ['CNN-LSTM', 'CNN-BiLSTM']):
        bar.set_color('orange')

plt.title('Model Size Comparison')
plt.ylabel('Size (KB)')
plt.xticks(rotation=45)

for i, v in enumerate(sizes.values()):
    plt.text(i, v, f'{v:.1f}KB', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 4. Compare Inference Times

In [ ]:
# Plot inference times
times = {name: m['inference_time']['mean'] for name, m in metrics.items()}
std_times = {name: m['inference_time']['std'] for name, m in metrics.items()}

plt.figure(figsize=(10, 6))
bars = plt.bar(times.keys(), times.values(),
               yerr=list(std_times.values()),
               capsize=5)

# Color hybrid models
for i, bar in enumerate(bars):
    if any(x in list(times.keys())[i] for x in ['CNN-LSTM', 'CNN-BiLSTM']):
        bar.set_color('orange')

plt.title('Model Inference Time Comparison')
plt.ylabel('Time (ms)')
plt.xticks(rotation=45)

for i, v in enumerate(times.values()):
    plt.text(i, v, f'{v:.1f}ms', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 5. Efficiency Analysis

In [ ]:
# Create comparison DataFrame
comparison_data = []
for name, m in metrics.items():
    data = {
        'Model': name,
        'Accuracy': m['accuracy'],
        'Size (KB)': m['model_size'],
        'Inference Time (ms)': m['inference_time']['mean'],
        'Memory Usage (MB)': m['memory_usage'],
        'Parameters': m['summary']['parameters'],
        'Efficiency Score': m['efficiency_score']
    }
    comparison_data.append(data)

df = pd.DataFrame(comparison_data)
print("Model Comparison:")
print(df.to_string(index=False))

# Plot efficiency scores
plt.figure(figsize=(10, 6))
bars = plt.bar(df['Model'], df['Efficiency Score'])

# Color hybrid models
for i, bar in enumerate(bars):
    if any(x in df['Model'].iloc[i] for x in ['CNN-LSTM', 'CNN-BiLSTM']):
        bar.set_color('orange')

plt.title('Model Efficiency Comparison')
plt.ylabel('Efficiency Score')
plt.xticks(rotation=45)

for i, v in enumerate(df['Efficiency Score']):
    plt.text(i, v, f'{v:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 6. Save Results

In [ ]:
# Save metrics
with open('results/model_metrics.pkl', 'wb') as f:
    pickle.dump(metrics, f)

# Save comparison DataFrame
df.to_csv('results/model_comparison_metrics.csv', index=False)

# Print final recommendations
print("\nModel Rankings by Efficiency:")
rankings = df.sort_values('Efficiency Score', ascending=False)
print(rankings[['Model', 'Efficiency Score', 'Accuracy', 'Size (KB)', 'Inference Time (ms)']]
      .to_string(index=False))

best_model = rankings.iloc[0]['Model']
print(f"\nBest performing model (efficiency-wise): {best_model}")

# Compare hybrid models
hybrid_models = df[df['Model'].str.contains('CNN-')]
print("\nHybrid Model Performance:")
print(hybrid_models[['Model', 'Efficiency Score', 'Accuracy', 'Size (KB)']]
      .to_string(index=False))